<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  








In [24]:
import pandas as pd
import numpy as np

def payment_data():
  
  #columns = ['Bankenclearing',	'ursprüngliche Mandanten-ID']#	Dokumentennummer	Zeitstempel	Vertrag	Vorname	Name	Produkt	ZREG	Vertragsverwaltungsuser	Service	PayNet-Nummer	Dokumentenschlüssel	Anzahl Seiten
  columns = ['contract_no', 'payment_amount']

  timestamp = pd.DataFrame()
  payment_id = pd.DataFrame(np.arange(1,100001), columns=['payment_id'])
  contract_no = pd.DataFrame(np.random.randint(1000001, size=100001), columns=['contract'])
  payment_amount = pd.DataFrame((np.random.sample(1000001)*50000), columns=['amount'])
  payment_amount = np.round_(payment_amount, decimals = 2)
  
  print(payment_amount)
 
  return

def transaction_checker():
  return
  
def session_checker():
  return

  #print(np.round_(payment_amount, decimals = 2))
payment_data()

#print (payment_id.shape, contract_no.shape, payment_amount.shape)

#pd.merge(payment_id, contract_no, payment_amount)



           amount
0        41633.73
1        22056.76
2        29328.93
3        44650.76
4        24587.83
5        41779.27
6        28046.32
7        30616.24
8        14015.30
9        23881.90
10       46277.12
11       29020.90
12        2357.53
13       27464.67
14       18181.00
15        7426.32
16       31222.36
17       12599.80
18       32110.45
19       39219.76
20       23135.86
21       14236.93
22       24825.79
23       43907.33
24        3126.25
25       24625.49
26       17271.45
27       47854.71
28       23476.72
29       15129.70
...           ...
999971   14352.81
999972   47848.53
999973   20643.68
999974    7680.35
999975    2233.52
999976   47429.73
999977   30337.39
999978   21061.68
999979   26842.49
999980   47493.67
999981   26174.75
999982   32659.88
999983    8571.59
999984   17947.25
999985   27550.41
999986   10276.21
999987   42378.34
999988    5237.55
999989   26972.49
999990   27099.20
999991   30400.73
999992   36066.19
999993   28244.11
999994   4